# Mission analysis

This section performs mission analysis for the example airplane. Similar to initial sizing, the [mission profile](../example_aircraft.md) is divided into various segments and the fuel fraction (or fuel weight) is estimated for each segment. However, the methods used in this section are more refined as compared to the methods used in intial sizing. The calculation incorporates additional information about the airplane computed in various preceding sections. For this demonstration, only the cruise and loiter estimation is refined. Other segments of the mission are assumed to be same as initial sizing. However, if refined models are available for other segments, then those should be used for computing fuel and weight fraction.

As described in [initial sizing](../initial_weight_estimation.ipynb), following table summarizes the weight fraction for each segment, except for cruise and loiter.

<div style="width:60%; margin: auto;">

Mission segment | Weight fraction | Value
------ | :-------: | :-------:
Warmup, taxi, takeoff | $W_1/W_0$ | 0.984
Climb | $W_2/W_1$ | 0.990
Descent | $W_4/W_3$ | 0.992
Climb | $W_5/W_4$ | 0.990
Descent | $W_7/W_6$ | 0.992
Land, taxi, shutdown | $W_8/W_7$ | 0.992
</div>

Note that above table does not specify a value for $W_3/W_2$ and $W_6/W_5$. Following table summarizes various fixed values used in this analysis.

<div style="width:100%; margin: auto;">

Parameter | Value | Source
-----| :----: | :-----:
Aspect ratio, $A$ | 8 | Assumed
Zero-lift drag coeff, $C_{D_0}$ | 0.033 | Calculated
Cruise propeller efficiency, $\eta_{p_{cr}}$ | 0.82 | Roskam Part 1, Table 2.2
Cruise brake specific fuel consumption, $C_{{bhp}_{cr}}$ | 0.4 lb/hr/bhp | Raymer, Table 3.4
Loiter propeller efficiency, $\eta_{p_{lo}}$ | 0.72 | Roskam Part 1, Table 2.2
Loiter brake specific fuel consumption, $C_{{bhp}_{lo}}$ | 0.4 lb/hr/bhp | Raymer, Table 3.4
</div>

Below code block imports requried packages and defines some variables:

In [1]:
import numpy as np

A = 8
CD0 = 0.03363 # clean configuration
prop_eff_cruise = 0.82
prop_eff_loiter = 0.72
Cbhp = 0.4 # lbs/hp-hr

# Weight fractions
W1_W0 = 0.985
W2_W1 = 0.99
W4_W3 = 0.992
W7_W6 = 0.992
W8_W7 = 0.992

Below sub-sections describe cruise and loiter analysis, and computes corresponding weight fractions. 

## Cruise

Similar to initial sizing section, the cruise analysis is performed using Breguet range equation but with some differences. In the initial sizing, the value of $C_L$ was computed such that $L/D$ and range is maximized. However, this does not take into account the cruise speed requirement of 200 kts. Hence, in this demonstration, $C_L$ is computed such that the cruise speed is requirement is met. The computed value of $C_L$ is then used to calculate $L/D$ using simple drag model described in [aerodynamics section](../aerodynamics/drag_polar_induced_drag.ipynb). Since the weight of the airplane decreases during cruise, the value of $C_L$ needs to be updated to maintain the cruise speed. Hence, the cruise segment is further divided into smaller segments and the weight is assumed to be constant in each of these segment. Thus, in this refined model, the weight fraction for cruise segment is computed as

$$
    \frac{W_3}{W_2} = \prod_{j=1}^{k} \frac{w_{j}}{w_{j-1}},
$$

$w_0 = W_2$

$$
    \frac{w_{j}}{w_{j-1}} = \exp \Bigg[ - \frac{ RC_{{bhp}_{cr}} }{ 375\eta_{p_{cr}} (L/D)_j } \Bigg]
$$

$$
    (L/D)_j = \frac{C_{L_j}}{C_{D_0} + KC_{L_j}^2}
$$

$$
    C_{L_j} = \frac{w_j}{0.5\rho V^2S}
$$

$$
    \frac{W_3}{W_2} = \exp \Bigg[ - \frac{ RC_{{bhp}_{cr}} }{ 375\eta_{p_{cr}} (L/D) } \Bigg]
$$



As a result, the $C_L$ is computed as

$$
    C_L = \frac{W}{0.5\rho V^2S},
$$

where $W$ is the weight of the airplane, $\rho$ is the air density at cruise altitude (8000 ft), and $V$ is the cruise speed. 

Hence, to refine the model, the 


In [ ]:
MTOW = 5374 # lbs

W1 = 0.984 * MTOW

W2 = 0.99 * W1

In [21]:
import numpy as np

C = 0.4 # lbs/hr/hp

num_segments = 5

fuel_fraction = []

total_range = 1200 * 1.15077945 # miles

individual_range = total_range / num_segments

prop_eff_cruise = 0.82

A = 8
e = 0.81
CD0 = 0.03363
S = 134 # sq ft

W_cruise = W2

V = 200 # knots
rho = 0.00186850 # slugs/ft^3
q = 0.5*rho*V**2

for i in range(num_segments):

    CL = W_cruise / q / S

    # print(CL)

    CD = CD0 + CL**2/np.pi/A/e

    T = q * S * CD / 0.92

    P = T * V / prop_eff_cruise / 550 # hp

    print(P)

    LbyD = CL/CD

    # print(LbyD)

    ff = np.exp( - individual_range * C / 375 / prop_eff_cruise / LbyD )

    print(ff)

    fuel_fraction.append(ff)

    W_cruise = W_cruise * fuel_fraction[-1]

    print(W_cruise)

weight_fraction = W_cruise / W2

print(weight_fraction)

210.76382950115712
0.9704388657104387
5080.379286409665
203.21546573297445
0.9706265224645088
4931.150879568536
196.15123599802513
0.9707871944455141
4787.098127763868
189.53177796323968
0.970921624805558
4647.89709231214
183.3217007226172
0.9710304321376526
4513.249522079196
0.8621074333152731


In [10]:
W3 = W_cruise

W4 = 0.992 * W3 # descent

W5 = 0.99 * W4 # climb

In [17]:
K = 1/A/np.pi/e
CL = (3*CD0/K)**0.5
CD = CD0 + K*CL**2
LbyD = CL/CD
E = 45/60 # hr

print(LbyD)

rho = 0.00211114 # 4000 ft, slugs/cu ft

fuel_fraction = []

W_loiter = W5

individual_E = E /num_segments

prop_eff_loiter = 0.72

for i in range(num_segments):

    V = (W_loiter/S * 2/rho * 1/CL )**0.5 * 0.682 # mph

    print(V)

    ff = np.exp( - individual_E * V * C / 375 / prop_eff_loiter / LbyD )

    print(ff)

    fuel_fraction.append(ff)

    W_loiter = W_loiter * fuel_fraction[-1]

    print(W_loiter)

weight_fraction = W_loiter / W5

print(weight_fraction)

10.653679846156203
97.84961324032776
0.9979610628833472
4164.2942746975405
97.74980773593917
0.9979631404568003
4155.812192163431
97.65020567686845
0.9979652137996026
4147.356002863374
97.55080644158379
0.9979672829246656
4138.925601498863
97.45160941108283
0.9979693478448487
4130.520883306169
0.9898673674461878


In [ ]:
W6 = W_loiter



## Power constraint

Cruise speed constraint is based on whether the engine can provide the thrust at that altitude

In [23]:
Pmax = 298*2 # hp

P_cruise = Pmax * 0.758 * 0.8

print(P_cruise)

361.41440000000006
